# Alloy blending problem


The company Steelco has received an order for 500 tonnes of steel to be used in shipbuilding.  The steel must have the following charactersitics:

| Chemical Element | Minimum Grade | Maximum Grade |
|------------------|---------------|---------------|
| Carbon (C)       | 2             | 3             |
| Copper (Cu)      | 0.4           | 0.6           |
| Manganese (Mn)   | 1.2           | 1.65          |

The company has seven different raw materials in stock that may be used for the production of this steel. The following table lists the grades, available amounts and prices for all materials:

| Raw Material | C%     | Cu%    | Mn%    | Availability in t | Cost in $/t  |
|--------------|--------|--------|--------|-------------------|--------------|
| Iron1        | 2.5    | 0      | 1.3    | 400               | 200          |
| Iron2        | 3      | 0      | 0.8    | 300               | 250          |
| Iron3        | 0      | 0.3    | 0      | 600               | 150          |
| Cu1          | 0      | 90     | 0      | 500               | 220          |
| Cu2          | 0      | 96     | 4      | 200               | 240          |
| Al1          | 0      | 0.4    | 1.2    | 300               | 200          |
| Al2          | 0      | 0.6    | 0      | 250               | 165          |

The objective is to determine the composition of the steel that minimizes the production cost.

#### Problem data

In [21]:
using JuMP

raw = [:iron1, :iron2, :iron3, :cu1, :cu2, :al1, :al2]

# composition (in percent) of [C, Cu, Mn]
composition = Dict(
:iron1 => [2.5,0,1.3],
:iron2 => [3,0,0.8],
:iron3 => [0,0.3,0],
:cu1 => [0,90,0],
:cu2 => [0,96,4],
:al1 => [0,0.4,1.2],
:al2 => [0,0.6,0])

# availability in tonnes
availability = Dict(
:iron1 => 400,
:iron2 => 300,
:iron3 => 600,
:cu1 => 500,
:cu2 => 200,
:al1 => 300,
:al2 => 250)

# cost in dollars per tonne
cost = Dict(
:iron1 => 200,
:iron2 => 250,
:iron3 => 150,
:cu1 => 220,
:cu2 => 240,
:al1 => 200,
:al2 => 165)

# minimum and maximum grade of [C, Cu, Mn]
MinGrade = [2, 0.4, 1.2]
MaxGrade = [3, 0.6, 1.65]
;

In [22]:
using Pkg
Pkg.build("HiGHS")
Pkg.add("HiGHS")
Pkg.add("ECOS")
Pkg.add("SCS")

   Resolving package versions...
  No Changes to `C:\Users\16084\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\16084\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\16084\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\16084\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\16084\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\16084\.julia\environments\v1.8\Manifest.toml`


In [55]:
# Question 1 (a)
using JuMP, HiGHS, ECOS, SCS
m = Model()
@variable(m, 0 <= x1 <= 4)
@variable(m, 0 <= x2 <= 4)
@variable(m, 0 <= x3 <= 4)

@objective(m, Max, 5x1 - x2 + 15x3)
@constraint(m, 2x1 >= x2 + 0.5x3)

set_optimizer(m, HiGHS.Optimizer)
@time optimize!(m)
optimize!(m)

println("Objective value: ", objective_value(m))
println("x1 = ", value(x1))
println("x2 = ", value(x2))
println("x3 = ", value(x3))

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-1); columns 0(-3); elements 0(-3) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  8.0000000000e+01
HiGHS run time      :          0.00
  0.001179 seconds (189 allocations: 12.766 KiB)
Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  8.0000000000e+01
HiGHS run time      :          0.00
Objective value: 80.0
x1 = 4.0
x2 = 0.0
x3 = 4.0


In [51]:
m = Model()
@variable(m, 0 <= x1 <= 4)
@variable(m, 0 <= x2 <= 4)
@variable(m, 0 <= x3 <= 4)

@objective(m, Max, 5x1 - x2 + 15x3)
@constraint(m, 2x1 >= x2 + 0.5x3)

set_optimizer(m, ECOS.Optimizer)
@time optimize!(m)
optimize!(m)

println("Objective value: ", objective_value(m))
println("x1 = ", value(x1))
println("x2 = ", value(x2))
println("x3 = ", value(x3))

  0.003064 seconds (985 allocations: 63.828 KiB)
Objective value: 79.9999999936439
x1 = 3.9999999999262186
x2 = 4.500795264542177e-9
x3 = 3.999999999900907

ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -3.807e+001  -1.499e+002  +1e+002  2e-003  3e-001  1e+000  1e+001    ---    ---    1  1  - |  -  - 
 1  -6.951e+001  -8.916e+001  +2e+001  3e-004  6e-002  1e+000  3e+000  0.9010  9e-002   0  0  0 |  0  0
 2  -7.790e+001  -9.311e+001  +1e+001  2e-004  6e-002  2e+000  2e+000  0.6129  4e-001   0  0  0 |  0  0
 3  -7.958e+001  -8.042e+001  +7e-001  1e-005  3e-003  1e-001  1e-001  0.9452  5e-004   0  0  0 |  0  0
 4  -8.000e+001  -8.001e+001  +9e-003  1e-007  4e-005  1e-003  1e-003  0.9881  3e-004   0  0  0 |  0  0
 5  -8.000e+001  -8.000e+001  +1e-004  2e-009  5e-007  2e-005  1e-005  0.9890  1e-004   1  0  0 |  0  0
 6  -8.000e+001  -8.000e+001  +1e-0

In [56]:
m = Model()
@variable(m, 0 <= x1 <= 4)
@variable(m, 0 <= x2 <= 4)
@variable(m, 0 <= x3 <= 4)

@objective(m, Max, 5x1 - x2 + 15x3)
@constraint(m, 2x1 >= x2 + 0.5x3)

set_optimizer(m, SCS.Optimizer)
@time optimize!(m)
optimize!(m)

println("Objective value: ", objective_value(m))
println("x1 = ", value(x1))
println("x2 = ", value(x2))
println("x3 = ", value(x3))

  0.003214 seconds (1.13 k allocations: 70.867 KiB)
Objective value: 80.00021048847219
x1 = 3.999847985380874
x2 = 0.00015128266468729975
x3 = 4.000074789615501
------------------------------------------------------------------
	       SCS v3.2.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 3, constraints m: 7
cones: 	  l: linear vars: 7
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 9, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|1.49e+002 1.11e+000 2.38e+003 -1.27e+003 1.

In [68]:
# For question 1 (a), they do not have same result.

In [67]:
#=
Question 1 (b)
The most fast one is ECOS, which is 0.000262 seconds
=#

In [69]:
#= 
Question 1 (c)
For 2x1 >= x2 + 0.5x3
It should be 2*4 >= 0 + 0.5*4 + α 
8 >= 2 + α
thus α = 6
=#

In [81]:
#=
Question 2 (a)

For x1=z1, x2=z2, x3=z3
max(-3z1 - 2z2 + z3)
subject to 2z1 - z2 + 3z3 = 10, 0 <= z2 <= 5, -5 <= z3 <= 20.

Let x = [z1, z2, z3, α, β, γ]
2z1 - z2 + 3z3 + α = 10
0 <= z2 + β = 5
-5 <= z3 + γ = 20

0 <= z1, z2, z3, α, β, γ (non-negative)
=#
print("A = [2 -1 3 1 0 0; 0 1 0 0 1 0; 0 0 1 0 0 1]\n")
print("b = [10 5 20]\n")
print("c = [-3 -2 1 0 0 0]")

A = [2 -1 3 1 0 0; 0 1 0 0 1 0; 0 0 1 0 0 1]
b = [10 5 20]
c = [-3 -2 1 0 0 0]

In [85]:
# Question 2(b)
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)

@variable(m, z1) 
@variable(m, 0 <= z2 <= 5) 
@variable(m, -5 <= z3 <= 20)

@constraint(m, 2z1 − z2 + 3z3 == 10) 
@objective(m, Min, 3z1 + 2z2 - z3)

optimize!(m)

println("objective = ", objective_value(m) )
println("z1 = ", value(z1) )
println("z2 = ", value(z2) )
println("z3 = ", value(z3) )

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
0 rows, 2 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-1); columns 0(-3); elements 0(-3) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     : -9.5000000000e+01
HiGHS run time      :          0.00
objective = -95.0
z1 = -25.0
z2 = 0.0
z3 = 20.0


In [95]:
#=
Question 3

x1 + x2 <= 1
x1 - x2 <= 1
-x1 - x2 <= 1
-x1 + x2 <= 1
x3 <= 2

A = [1, 1, 0]
b = [1, 1, -1, -1, 2]
=#
print("A = [1 1 0; 1 -1 0; -1 -1 0; -1 1 0; 0 0 1]\n")
print("b = [1, 1, -1, -1, 2]")

A = [1 1 0; 1 -1 0; -1 -1 0; -1 1 0; 0 0 1]
b = [1, 1, -1, -1, 2]

In [2]:
# Question 4 (a)
using JuMP
raw = [:iron1, :iron2, :iron3, :cu1, :cu2, :al1, :al2]

# composition (in percent) of [C, Cu, Mn]
composition = Dict(
:iron1 => [2.5,0,1.3],
:iron2 => [3,0,0.8],
:iron3 => [0,0.3,0],
:cu1 => [0,90,0],
:cu2 => [0,96,4],
:al1 => [0,0.4,1.2],
:al2 => [0,0.6,0])

# availability in tonnes
availability = Dict(
:iron1 => 400,
:iron2 => 300,
:iron3 => 600,
:cu1 => 500,
:cu2 => 200,
:al1 => 300,
:al2 => 250)

# cost in dollars per tonne
cost = Dict(
:iron1 => 200,
:iron2 => 250,
:iron3 => 150,
:cu1 => 220,
:cu2 => 240,
:al1 => 200,
:al2 => 165)

# minimum and maximum grade of [C, Cu, Mn]
MinGrade = [2, 0.4, 1.2]
MaxGrade = [3, 0.6, 1.65]
;

using JuMP, HiGHS

m = Model(HiGHS.Optimizer)
@variable(m, 0 <= iron1 <= availability[:iron1])
@variable(m, 0 <= iron2 <= availability[:iron2])
@variable(m, 0 <= iron3 <= availability[:iron3])
@variable(m, 0 <= cu1 <= availability[:cu1])
@variable(m, 0 <= cu2 <= availability[:cu2])
@variable(m, 0 <= al1 <= availability[:al1])
@variable(m, 0 <= al2 <= availability[:al2])

x = composition[:iron1]*iron1 + composition[:iron2]*iron2 + composition[:iron3]*iron3 + composition[:cu1]*cu1 + composition[:cu2]*cu2 + composition[:al1]*al1 + composition[:al2]*al2
@constraint(m, (iron1 + iron2 + iron3 + cu1 + cu2 + al1 + al2) == 500)
@constraint(m, x[1] >= MinGrade[1]*500)
@constraint(m, x[1] <= MaxGrade[1]*500)
@constraint(m, x[2] >= MinGrade[2]*500)
@constraint(m, x[2] <= MaxGrade[2]*500)
@constraint(m, x[3] >= MinGrade[3]*500)
@constraint(m, x[3] <= MaxGrade[3]*500)

@objective(m, Min, cost[:iron1]*iron1 + cost[:iron2]*iron2 + cost[:iron3]*iron3 + cost[:cu1]*cu1 + cost[:cu2]*cu2 + cost[:al1]*al1 + cost[:al2]*al2)

optimize!(m)
print("Min: ", objective_value(m))

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
7 rows, 7 cols, 29 nonzeros
4 rows, 7 cols, 18 nonzeros
4 rows, 7 cols, 18 nonzeros
Presolve : Reductions: rows 4(-3); columns 7(-0); elements 18(-11)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 4(2200) 0s
          3     9.8121635792e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 3
Objective value     :  9.8121635792e+04
HiGHS run time      :          0.01
Min: 98121.63579168123